In [1]:
## The usual imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
## for printing image
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np

In [2]:
print(torch.__version__)

1.1.0.post2


In [3]:
# 如果GPU可用将使用GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else"cpu") 

### 导入并准备数据

In [4]:
## parameter denoting the batch size
# 并读取图像文件
BATCH_SIZE = 32 
jpg_path = "./data/GTSRB_train_jpgs/"
files = [file for file in os.listdir(jpg_path) if ".jpg" in file]
std_size = (28, 28)
images = [cv2.cvtColor(cv2.resize(cv2.imread(jpg_path + file), std_size), cv2.COLOR_BGR2GRAY).reshape(1, std_size[1], std_size[0]) for file in files]
labels = [int(file.split("#")[0]) for file in files]

In [5]:
# 将读取到的图像文件转化为Tensor 但是当前的训练数据 tensor是整数类型的 
# 不能直接用于训练需要在后续步骤中转化为float类型 
images_tensor = torch.tensor(images)
labels_tensor = torch.tensor(labels)

labels.dtypes

In [6]:
# 编写一个生成器 用于逐个batch 将数据迭代产生 
# 此处注意数据的类型 用来训练的图像数据要转化为 torch.FloatTensor (float32)
# 标签要转化为 torch.LongTensor (int64)
def train_data_loader(train_set, labels, batch_size):
    for i in range(0, train_set.shape[0], batch_size):
        batch_imgs = train_set[i: i + batch_size].type(torch.FloatTensor)
        batch_labels = labels[i: i + batch_size].type(torch.LongTensor)
        yield (batch_imgs, batch_labels)

# 卷积层的输入尺寸为 (Batch_size, Channel, Height, Width) 对于灰度图像 Channel为1 RGB图像为1 
# 对于其他卷积层产生的输入 Channel 为上一层卷积核的个数
# 使用view函数对矩阵的形状进行变换
# 所以上述的输入为
# torch.Size([32, 1, 28, 28])
def my_loader():
    return train_data_loader(images_tensor.view(images_tensor.shape[0], 1, 28, 28), labels_tensor, 32)

In [7]:
# 查看一下迭代的结果
for images, labels in my_loader(): 
    print("Image batch dimensions:", images.shape, images.dtype) 
    print("Image label dimensions:", labels.shape, labels.dtype) 
    print(labels)
    break
print(type(my_loader))

Image batch dimensions: torch.Size([32, 1, 28, 28]) torch.float32
Image label dimensions: torch.Size([32]) torch.int64
tensor([12,  9, 25,  7,  1, 10, 23, 35,  2, 13, 28,  4, 11, 17,  2, 35, 33, 11,
        25,  9, 12,  1, 38, 42, 25,  3,  9, 12,  7,  1, 10, 42])
<class 'function'>


### 继承nn.Module 模块定义一个神经网络的前馈结构
- 在torch中 定义网络的前馈结构即可 
- 需要继承nn.Module 
- 在__init__ 中定义网络的常量 以及 各个层的尺寸 
- 定义一个层如下   
```self.c3 = torch.nn.Conv2d(6, 16, (5, 5), stride=1, padding=0, bias=True, padding_mode='zeros')```    
   实际上是定义了一个偏函数, 偏函数包括了卷积尺寸，核数量等信息 但是就是 没有输入数据x
- forward中定义 网络真正的结构 使用__init__中事先定义好的偏函数 定义各个偏函数与输入和输出的关系 
- 构建过程相当于写下一个大号的公式 forward 返回最终通过网络的输出

In [8]:
class MyModel(nn.Module): 
    def __init__(self): 
        # 在init中定义 各层对象结构
        super(MyModel, self).__init__() 
        self.class_num = 43
        # 就是一般的全连接层 第一个参数是输入维度 第二个参数是输出维度
        # 注意卷积层的输入为尺寸为 (N, Channel, Height, Width)
        # 
        self.c1 = torch.nn.Conv2d(1, 6, (5, 5), stride=1, padding=0, bias=True, padding_mode='zeros')
        self.s2 = torch.nn.MaxPool2d((2, 2))
        self.c3 = torch.nn.Conv2d(6, 16, (5, 5), stride=1, padding=0, bias=True, padding_mode='zeros')
        self.s4 = torch.nn.MaxPool2d((2, 2))
        self.d1 = nn.Linear(256, 120) 
        #self.dropout = nn.Dropout(p=0.2) 
        self.d2 = nn.Linear(120, 84) 
        self.d3 = nn.Linear(84, self.class_num)
        
    def forward(self, x): 
        # 使用init中创建的层对象 构建网络的连接关系
        #print("input_shape", x.shape)
        x =  F.relu(self.s2(self.c1(x)))
        x =  F.relu(self.s4(self.c3(x)))
        x = x.view(-1, 256) # 把3d(depth,x,y)的图像数据展平为一维的 这里用的就是普通神经网络不是CNN
        x = F.relu(self.d1(x))  # 全连接层1
        x = F.relu(self.d2(x))  # 全连接层1
        x = self.d3(x)
        out = F.log_softmax(x, dim=1)
        #exit()
        return out
        #return out

### 开始训练模型
- 训练模型就是通过比对模型的输出和真实的输出之间的差距 然后使用梯度下降法调整模型参数 减小这种差距
- 首先除了上面一步定义好的模型之外 还要定义损失函数 也就是衡量模型输出和真实值之间差距的函数 
  损失函数的输出就是 loss 我们的目标就是优化loss
- 优化器optimizer 可以看做是一个包含了模型参数的容器 在backward之后 可以操作 更新模型中的参数 

In [9]:
learning_rate = 0.001 
num_epochs = 5 
device = torch.device("cuda:0" if torch.cuda.is_available() else"cpu") 
model = MyModel() 
model = model.to(device) # 模型放到CPU 或者GPU上
criterion = nn.CrossEntropyLoss() # 定义损失函数
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

- 自己写一个计算准确度的函数
- 计算当前的batch中的准确度 torch.max(output, 1) 中的第二个参数表示在第1个维度(维度编号从0开始)中求最大值 
- 该函数除了返回对应的最大值还会返回对应维度中的最大值所在的index 很方便 下面是返回示例   

```
torch.return_types.max(
values=tensor([1.3117, 2.0132, 1.4627, 1.4121, 1.5152, 1.5220, 1.8655, 2.3365, 1.8300,
        1.2343], dtype=torch.float64),
indices=tensor([0, 5, 9, 7, 6, 0, 4, 6, 7, 8]))
```

In [10]:
def get_accuracy(output, target, batch_size):
    ''' Obtain accuracy for training round '''
    corrects = (torch.max(output, 1)[1].view(target.size()).data == target.data).sum() 
    accuracy = 100.0 * corrects/batch_size 
    return accuracy.item()

- 定义好迭代的轮数 在每一轮训练中要吧 
- 当前优化器中的梯度清零 CNN中 不能累加梯度  RNN中可能有需要 
  [zero_grad的解释](https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
- 每轮训练遵循的流程都是 输入数据->神经网络模型->计算loss->backward()->optimizer.step()

In [11]:
## train the model 
num_epochs = 5
for epoch in range(num_epochs): 
    train_running_loss = 0.0 
    train_acc = 0.0 
     ## commence training 
    model = model.train() 
     ## training step 
    #for i, (images, labels) in enumerate(train_data_loader()): 
    for i, (images, labels) in enumerate(my_loader()):
        images = images.to(device) 
        labels = labels.to(device) 
         ## forward + backprop + loss 
        predictions = model(images) 
        #print(predictions, predictions.shape)
        #print(labels, labels.shape)
        #break
        loss = criterion(predictions, labels) 
        optimizer.zero_grad() # 梯度清零
        loss.backward()
        ## update model params 
        optimizer.step() # 更新模型参数 
        train_running_loss += loss.detach().item()  # 累加loss时记得断开求导 不然会变成求累加loss的导数
        train_acc += get_accuracy(predictions, labels, BATCH_SIZE) 
    model.eval()
    print('Epoch: %d | Loss: %.4f | Train Accuracy: %.2f' %(epoch,train_running_loss / i, train_acc/i))

Epoch: 0 | Loss: 0.8793 | Train Accuracy: 77.51
Epoch: 1 | Loss: 0.2446 | Train Accuracy: 93.33
Epoch: 2 | Loss: 0.1621 | Train Accuracy: 95.43
Epoch: 3 | Loss: 0.1355 | Train Accuracy: 95.98
Epoch: 4 | Loss: 0.1154 | Train Accuracy: 96.51


- 输入测试数据进行预测
- 可以看出就是直接把数据灌入到model中就可以了 
- 为了方便下面的例子只预测了一个batch中的准确率

In [12]:
test_acc = 0.0 
labels = None
outputs = None
for i, (images, labels) in enumerate(my_loader()): 
    images = images.to(device) 
    labels = labels.to(device) 
    outputs = model(images)
    break
print("1 batch test_acc", get_accuracy(outputs, labels, BATCH_SIZE))

1 batch test_acc 100
